<a href="https://colab.research.google.com/github/putriayu02/Sortify-Bangkit-Capstone/blob/validation_test_modelling/Analisis_Model_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Model CNN - Klasifikasi Sampah


Notebook ini digunakan untuk menguji apakah model klasifikasi sampah mengalami overfit, underfit, atau masalah lain. Notebook ini mencakup:
1. Memuat model.
2. Mengevaluasi performa model.
3. Visualisasi performa.
4. Live detection menggunakan upload file.


In [1]:

# Import library yang diperlukan
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import os
from google.colab import files
from PIL import Image

# Pastikan Anda menggunakan TensorFlow versi 2.x
print(f"TensorFlow version: {tf.__version__}")


TensorFlow version: 2.17.1


In [2]:

# Load model yang sudah Anda miliki
model_path = "model_capstonelancar.h5"  # Path file model Anda
model = load_model(model_path)
model.summary()  # Menampilkan arsitektur model


OSError: Unable to synchronously open file (truncated file: eof = 12582912, sblock->base_addr = 0, stored_eof = 307607904)

## Evaluasi Performa Model pada Dataset Training dan Testing

In [ ]:

# Path dataset
train_dir = "path_to_train_dataset"  # Ganti dengan path dataset training
test_dir = "path_to_test_dataset"  # Ganti dengan path dataset testing

# Preprocessing data menggunakan ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Sesuaikan dengan ukuran input model Anda
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Evaluasi pada training dan testing data
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print(f"Training Accuracy: {train_acc:.2f}, Training Loss: {train_loss:.2f}")
print(f"Testing Accuracy: {test_acc:.2f}, Testing Loss: {test_loss:.2f}")


## Visualisasi Performa Model

In [ ]:

# Plot akurasi dan loss selama training
history_path = "training_history.npy"  # Jika Anda menyimpan history training sebagai file .npy

if os.path.exists(history_path):
    history = np.load(history_path, allow_pickle=True).item()
    plt.figure(figsize=(12, 4))

    # Plot akurasi
    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('Akurasi Model')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss Model')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()
else:
    print("History training tidak ditemukan. Pastikan Anda menyimpan history saat training.")


## Live Detection dengan Upload File

In [ ]:

# Upload file gambar untuk prediksi langsung
uploaded = files.upload()

for filename in uploaded.keys():
    # Load gambar
    img = Image.open(filename)
    img = img.resize((150, 150))  # Sesuaikan ukuran dengan input model
    img_array = np.array(img) / 255.0  # Normalisasi
    img_array = np.expand_dims(img_array, axis=0)  # Menambahkan batch dimension

    # Prediksi
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    class_labels = list(train_generator.class_indices.keys())
    print(f"Prediksi untuk {filename}: {class_labels[predicted_class[0]]}")
